In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [3]:
sqlContext.sql("use dsc10742_gcctmsd_lz_db")

DataFrame[]

In [5]:
# For Prod data
import xlrd
old=pd.ExcelFile('/s/latluri1/DES/GCCT_schema_Jan19.xlsx').parse("sheet1")
new=pd.ExcelFile('GCCT_schema_Mar2019.xlsx').parse("sheet1")

In [4]:
# For QA
new=pd.read_csv('/s/latluri1/GCCT_DES_DSC_monitoring/PPROD/archive/GCCT_schema_DSC_QA_Aug_19_2019_LZ.csv')
old=pd.read_csv('/s/latluri1/GCCT_DES_DSC_monitoring/PPROD/archive/GCCT_schema_DSC_QA_June_11_2019_LZ.csv')

In [5]:
final=old.merge(new,how='outer',on=['table','col_name'],suffixes=('_old','_new') )
final

,col_name,data_type_old,table,data_type_new
0,account_identity,bigint,account,bigint
1,account_rowversion,string,account,string
2,id,string,account,string
3,sinkcreatedon,timestamp,account,timestamp
4,sinkmodifiedon,timestamp,account,timestamp
5,statecode,int,account,int
6,statuscode,int,account,int
7,customertypecode,int,account,int
8,address1_freighttermscode,int,account,int
9,address1_addresstypecode,int,account,int


# ADDED

In [6]:
final[final.data_type_old.isnull()]

,col_name,data_type_old,table,data_type_new
12648,gcct_defaultportalcasetypeid,NaN,contact,string
12649,gcct_portalcasetypeid,NaN,gcct_caseclassification,string
12650,gcct_ recursivecounter,NaN,gcct_vehicleowner,string
12651,gcct_integrationstatus,NaN,gcct_vehicleowner,int
12652,gcct_caseworkedby,NaN,incident,string
12653,gcct_activitystageoptioncode,NaN,gcct_catalogcorrection,int
12654,gcct_activitystageoptioncodename,NaN,gcct_catalogcorrection,string
12655,gcct_assigntoteamid,NaN,gcct_catalogcorrection,string
12656,gcct_bodystyleid,NaN,gcct_catalogcorrection,string
12657,gcct_catalogdeckid,NaN,gcct_catalogcorrection,string


In [1]:
#gcct_counterofferedbuybacktype is int in infor schema



# Removed

In [7]:
final[final.data_type_new.isnull()]

,col_name,data_type_old,table,data_type_new
32,gcct_accountstatus,int,account,NaN


# Dtype changed

In [8]:
final.loc[(final.data_type_new!=final.data_type_old) & (np.logical_not(final.data_type_new.isnull())) & (np.logical_not(final.data_type_old.isnull()))]

,col_name,data_type_old,table,data_type_new


In [ ]:
#gcct_vehicleoffroad is int
#gcct_customerinrental in int